# **Homework**: Open source data ingestion for RAGs with dlt

Video: https://www.youtube.com/watch?v=qUNyfR_X2Mo

In this hands-on workshop, we’ll learn how to build a data ingestion pipeline using dlt to load data from a REST API into LanceDB so you can have an always up to date RAG.

​We’ll cover the following steps:

* Extract data from REST APIs
* Loading and vectorizing into LanceDB, which unlike other vector DBs stores the data _and_ the embeddings
* Incremental loading

​By the end of this workshop, you’ll be able to write a portable, OSS data pipeline for your RAG that you can deploy anywhere, such as python notebooks, virtual machines, or orchestrators like Airflow, Dagster or Mage.

## Resources

* Slides: [dlt-LLM-Zoomcamp.pdf](https://github.com/user-attachments/files/16131729/dlt.LLM.Zoomcamp.pdf)
* [Google Colab notebook](https://colab.research.google.com/drive/1nNOybHdWQiwUUuJFZu__xvJxL_ADU3xl?usp=sharing) - make a copy to follow along!

---

## Homework

In the workshop, we extracted contents from two pages in notion titled "Workshop: Benefits and Perks" and "Workshop: Working hours, PTO, and Vacation". 

Repeat the same process for a third page titled "Homework: Employee handbook" (hidden from public view, but accessible via API key):

1. Modify the REST API source to extract only this page.
2. Write the output into a separate table called "homework".
3. Remember to update the table name in all cells where you connect to a lancedb table.

To do this you can use the [workshop Colab](https://colab.research.google.com/drive/1nNOybHdWQiwUUuJFZu__xvJxL_ADU3xl?usp=sharing) as a basis.

Now, answer the following questions:

## Q1. Rows in LanceDB

How many rows does the lancedb table "notion_pages__homework" have?

* 14
* 15
* 16
* 17

### Notes:

:white_check_mark: 17

### 4. Write the pipeline code

**Note**: We first go over the code step by step before putting it into runnable cells

1. Import necessary modules (run this cell)

In [1]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

2. Configure the dlt rest api source to connect to and extract the relevant data out from the Notion REST API.

    Our notion space has multiple pages ...

---

  Note on pagination:

  Different REST APIs might use different strategies to handle paginated responses. dlt has built-in support for [most common pagination mechanisms](https://dlthub.com/docs/general-usage/http/rest-client#paginators), and these can be explicity passed inside the configuration like shown above.

  However in most cases, it won't be necessary to explicity specify the pagination strategy, since dlt detects it automatically.

  In case the specific pagination is not supported by dlt yet, then you can also implement a custom paginator. For example, dlt does not have a built-in paginator for POST methods, so we write our own paginator. We take the [code provided in the docs for it](https://dlthub.com/docs/general-usage/http/rest-client#example-2-creating-a-paginator-for-post-requests), and make small modifications to it based on the [notion API documentation](https://developers.notion.com/reference/intro#parameters-for-paginated-requests).

In [2]:
class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

3. Extract relevant content from the response body

  The response returned from the API is a nested JSON which we need to pre-process before using it anywhere. dlt can unnest json automatically, but since the Notion API is a little tricky it's better to pre-process this first so we have a more structured DB as a result.
  
  One way to do this is to pass the JSON response through a transformation function that will extract only the relevant data from the JSON body (we later add this as a mapping to the resource):

In [3]:
# Added imports
from datetime import datetime, timezone

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

### 5. Run the pipeline

Run this block:

In [4]:
# from datetime import datetime, timezone

# class PostBodyPaginator(BasePaginator):
#     ...


# @dlt.resource(name="employee_handbook")
@dlt.resource(name="homework")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        # "query": "workshop",
                        "query": "homework",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    # yield from rest_api_source(notion_config,name="employee_handbook")
    yield from rest_api_source(notion_config,name="homework")

# def extract_page_content(response):
#     ...


@dlt.resource(
    # name="employee_handbook",
    name="homework",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    last_value = last_edited_time.last_value
    print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        # table_name="employee_handbook",
        table_name="homework",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

c:\Users\Administrator\Projects\rye-demo\.venv\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-07-05T23:33:00.000Z
Pipeline company_policies load step completed in ---
0 load package(s) were loaded to destination LanceDB and into dataset None
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x000002187CAE0770> location to store data


In [5]:
import lancedb


db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework")

homework = dbtable.to_pandas()
print(f"{homework.shape=}")
print('Q1 Answer: The lancedb table "notion_pages__homework" have {} rows'.format(homework.shape[0]))

homework.shape=(17, 9)
Q1 Answer: The lancedb table "notion_pages__homework" have 17 rows


## Q2. Running the Pipeline: Last edited time

In the demo, we created an incremental dlt resource `rest_api_notion_incremental` that keeps track of `last_edited_time`. What value does it store after you've run your pipeline once? (Hint: you will be able to get this value by performing some aggregation function on the column `last_edited_time` of the table)

* `Timestamp('2024-07-05 22:34:00+0000', tz='UTC') (OR "2024-07-05T22:34:00.000Z")`
* `Timestamp('2024-07-05 23:33:00+0000', tz='UTC') (OR "2024-07-05T23:33:00.000Z")`
* `Timestamp('2024-07-05 23:52:00+0000', tz='UTC') (OR "2024-07-05T23:52:00.000Z")`
* `Timestamp('2024-07-05 22:56:00+0000', tz='UTC') (OR "2024-07-05T22:56:00.000Z")`

### Notes:

:white_check_mark: `Timestamp('2024-07-05 23:33:00+0000', tz='UTC') (OR "2024-07-05T23:33:00.000Z")`

In [6]:
print(homework["last_edited_time"])

last_edited_time = homework["last_edited_time"].max()
print(f"Q2 Answer: {last_edited_time=}")

0    2024-07-05 22:34:00+00:00
1    2024-07-05 22:38:00+00:00
2    2024-07-05 22:52:00+00:00
3    2024-07-05 22:52:00+00:00
4    2024-07-05 22:52:00+00:00
5    2024-07-05 22:52:00+00:00
6    2024-07-05 22:52:00+00:00
7    2024-07-05 22:52:00+00:00
8    2024-07-05 23:33:00+00:00
9    2024-07-05 22:52:00+00:00
10   2024-07-05 22:52:00+00:00
11   2024-07-05 22:52:00+00:00
12   2024-07-05 22:52:00+00:00
13   2024-07-05 22:52:00+00:00
14   2024-07-05 22:52:00+00:00
15   2024-07-05 22:54:00+00:00
16   2024-07-05 22:56:00+00:00
Name: last_edited_time, dtype: datetime64[us, UTC]
Q2 Answer: last_edited_time=Timestamp('2024-07-05 23:33:00+0000', tz='UTC')


## Q3. Ask the Assistant 

Find out with the help of the AI assistant: how many PTO days are the employees entitled to in a year?  

* 20
* 25
* 30
* 35

### Notes:

:white_check_mark: 30

5. Write a function that can retrieve content from lancedb relevant to the user query
  
  With LanceDB, you don't have to explicity embed the question. LanceDB stores information on the embedding model used and automatically embeds the question.

  We use the `db_table.search()` function to query the DB and then limit it to the top 2 most similar results and return that as the context to pass to the RAG.

  Limiting results is important because otherwise there might be too much confusing information. Similarly only picking the top choice might not give enough information.

In [7]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

6. Finally we define a very basic RAG. We define a simple system prompt, retrieve the relevant context for the user query with the function defined above and then send the user question and the context to the `llama2-uncensored` model.

In [8]:
# Added imports
import ollama


def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
#   dbtable = db.open_table("notion_pages___employee_handbook")
  dbtable = db.open_table("notion_pages___homework")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        # model="llama2-uncensored",
        model="gemma:2b",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

And we run the RAG! Some example questions you can ask:

* How many vacation days do I get?
* Can I get maternity leave?

**Note**: This is a very basic implementation of a RAG, since this workshop is mainly about data ingestion. So expect some weird answers. If you do stop and restart the cell, you will need to rerun the cell containing `ollama serve` first.

In [9]:
main()

Assistant: Okay, I understand that employees receive 30 days of PTO per year. However, the context does not specify how many of these 30 days can be taken at any given time during the year. Could you please provide more information about this?
